In [ ]:
# Tuddal coordinates: 59.7463 N  8.8083 E

# Constants

epsilon_surface = 0.97  # Emissivity of the surface
sigma = 5.67e-8  # Stefan-Boltzmann constant (W/m^2 K^4)

lambda_vaporization = 2.5e6  # Latent heat of vaporization (J/kg)
rho_air = 1.225  # Air density (kg/m^3)
C_e = 0.0015  # Bulk transfer coefficient for latent heat

Cp = 1005  # Specific heat capacity of air at constant pressure (J/(kg*K))
Ch = 0.001  # Bulk transfer coefficient for sensible heat
#––––––––––––––––––

alpha_snow = 0.8  # Albedo for snow-covered surface
alpha_veg = 0.15  # Albedo for shrubs

In [1]:
import numpy as np

def surface_albedo(snow_cover_fraction):
    # snow cover fraction spans [0,1]
    return snow_cover_fraction * (alpha_snow - alpha_veg) + alpha_veg #rescale the snow_fraction to [0.15-0.8]

# Function for outgoing shortwave radiation
def SW_out(SWin, snow_cover_fraction):
    albedo = surface_albedo(snow_cover_fraction)
    SW_out = albedo * SW_in
    return SW_out

# Function for outgoing longwave radiation
def LW_out(T_s): # Ts is in K
    LW_out = epsilon_surface * sigma * (T_s)**4
    return LW_out

# Function for sensible heat flux
def SH(T_s, T_a, u):
    SH = rho_air * Cp * Ch * u * (T_s - T_a)
    return SH

# Function for latent heat flux
def LE(q_s, q_a, u):
    LE = lambda_vaporization * rho_air * C_e * u * (q_s - q_a)
    return LE

Outgoing Shortwave Radiation (SW_out): 160.0
Outgoing Longwave Radiation (LW_out): 284.3583583822426
Sensible Heat Flux (SH): -12.311250000000001
Cloud Fraction (CF): 0.47769821806997037
Incoming Longwave Radiation (LW_in): 330.849927735018
Latent Heat Flux (LE): 1.8612790657335032
Updated Surface Temperature (T_s): 343.98954503335085


In [ ]:
# Example scenario variables
T_s_example = -5  # Initial surface temperature in Celsius
T_a_example = -3  # Air temperature in Celsius
SW_in_example = 200  # Example incoming shortwave radiation in W/m^2
u_example = 5  # Wind speed in m/s
q_a_example = 0.002  # Specific humidity in the air (example)
C_s_example = 1000  # Surface heat capacity in J/(m^2*K)
snow_cover_example = 1  # Snow presence

# Calculations
SW_out_example = outgoing_shortwave_radiation(SW_in_example, snow_cover_example)
LW_out_example = outgoing_longwave_radiation(T_s_example)
SH_example = sensible_heat_flux(T_s_example, T_a_example, u_example)
CF_example, LW_in_example = cloud_fraction_and_LW_in(T_a_example)
LE_example = latent_heat_flux(surface_specific_humidity(T_s_example, snow_cover_example), q_a_example, u_example)
T_s_updated_example = update_surface_temperature(T_s_example, SW_in_example, LW_in_example, LE_example, SH_example, snow_cover_example, C_s_example)

# Display results
print("Outgoing Shortwave Radiation (SW_out):", SW_out_example)
print("Outgoing Longwave Radiation (LW_out):", LW_out_example)
print("Sensible Heat Flux (SH):", SH_example)
print("Cloud Fraction (CF):", CF_example)
print("Incoming Longwave Radiation (LW_in):", LW_in_example)
print("Latent Heat Flux (LE):", LE_example)
print("Updated Surface Temperature (T_s):", T_s_updated_example)


In [ ]:
# Function to calculate saturation specific humidity
def saturation_specific_humidity(T):
    """Calculate saturation specific humidity (in kg/kg) at temperature T (Celsius)."""
    e_s = 6.112 * np.exp((17.67 * T) / (T + 243.5))  # Saturation vapor pressure in hPa
    q_sat = 0.622 * (e_s / (1013.25 - e_s))  # Specific humidity formula
    return q_sat

# Function to calculate specific humidity at the surface (q_s)
def surface_specific_humidity(T_s, snow_cover):
    """Calculate specific humidity at the surface based on temperature and snow cover."""
    q_sat_surface = saturation_specific_humidity(T_s)
    if snow_cover == 1:  # Snow-covered
        q_s = q_sat_surface * alpha_snow
    else:  # Snow-free (soil/water)
        q_s = q_sat_surface * alpha_soil_water
    return q_s
    
# Stochastic cloud fraction and LW_in calculation
def cloud_fraction_and_LW_in(T_a):
    CF = np.random.beta(alpha_cloud, beta_cloud)
    LW_clear = sigma * (T_a + 273.15)**4
    LW_cloudy = 1.2 * LW_clear
    LW_in = (1 - CF) * LW_clear + CF * LW_cloudy
    return CF, LW_in

# Surface temperature update based on energy balance
def update_surface_temperature(T_s, SW_in, LW_in, LE, SH, snow_cover, C_s, time_step=3600):
    SW_out = outgoing_shortwave_radiation(SW_in, snow_cover)
    LW_out = outgoing_longwave_radiation(T_s)
    net_radiation = SW_in - SW_out + LW_in - LW_out - LE - SH
    dT_s = (net_radiation / C_s) * time_step
    T_s_updated = T_s + dT_s
    return T_s_updated

In [4]:
import datetime

def estimate_surface_temp(air_temp, snow_cover_fraction, date):
    """
    Estimate surface temperature based on air temperature, snow cover fraction, and date.
    
    :param air_temp: Air temperature in Celsius
    :param snow_cover_fraction: Snow cover fraction (0 to 1)
    :param date: Date in 'YYYY-MM-DD' format
    :return: Estimated surface temperature in Celsius
    """
    # Convert date string to datetime object
    date = datetime.datetime.strptime(date, '%Y-%m-%d')
    
    # Define seasons
    spring = range(3, 6)
    summer = range(6, 9)
    autumn = range(9, 12)
    winter = [1,2,12]

    # Adjust temperature based on season
    if date.month in summer:
        season_adjust = 2  # Surface tends to be warmer than air in summer
    elif date.month in winter:
        season_adjust = -1  # Surface tends to be colder than air in winter
    else:
        season_adjust = 0  # Spring and autumn are transitional

    # Calculate base surface temperature
    base_surface_temp = air_temp + season_adjust

    # Adjust for snow cover
    if snow_cover_fraction > 0:
        # Snow-covered surface temperature (assuming it's close to 0°C when there's snow)
        snow_surface_temp = min(0, air_temp)  # Can't be above 0°C, but can be below in very cold conditions
        
        # Weighted average between snow-covered and snow-free surface
        surface_temp = (snow_cover_fraction * snow_surface_temp + 
                        (1 - snow_cover_fraction) * base_surface_temp)
    else:
        surface_temp = base_surface_temp

    # Add a small random variation to account for local factors (-0.5 to 0.5°C)
    surface_temp += (datetime.datetime.now().microsecond / 1000000) - 0.5

    return round(surface_temp, 1)

# Example usage:
air_temp = 5  # °C
snow_cover = 0.3  # 30% snow cover
date = '2024-11-14'  # Current date from your previous message

estimated_temp = estimate_surface_temp(air_temp, snow_cover, date)
print(f"Estimated surface temperature: {estimated_temp}°C")

Estimated surface temperature: 3.4°C
